In [ ]:
# ------------------------------------------------------------------------------
# DOCUMENT GENERATION WITH GUIDELINES
# ------------------------------------------------------------------------------
def generate_document_with_guidelines(
    user_query: str,
    guidelines: List[str],
    vector_store: VectorStore,
    k: int = 3,
    max_new_tokens: int = 512,
    temperature: float = 0.7
) -> str:
    """
    Searches the VectorStore for relevant doc chunks based on user_query,
    then iterates over each guideline to produce a separate snippet.
    Returns one combined "document" with all guideline-based responses.
    """
    # 1) Retrieve top-k relevant documents
    search_results = vector_store.search(user_query, k=k)

    # 2) Build a single combined context from the top docs
    combined_context_parts = []
    for doc, sim_score in search_results:
        meta_str = ""
        if doc.metadata:
            meta_parts = [f"{key}: {val}" for key, val in doc.metadata.items()]
            meta_str = "\n".join(meta_parts)

        context_str = (
            f"---\nContent:\n{doc.content}\n"
            f"Metadata:\n{meta_str}\n"
            f"Similarity Score: {sim_score:.4f}\n---"
        )
        combined_context_parts.append(context_str)
    combined_context = "\n\n".join(combined_context_parts)

    # 3) Loop over each guideline, generate a snippet
    final_snippets = []
    for idx, guideline in enumerate(guidelines, start=1):
        prompt = f"""
You have the following user query:
{user_query}

Context from relevant documents (with metadata):
{combined_context}

Guideline #{idx}: {guideline}

Based on the user query and the above context, create a concise 
section of a final document that follows this guideline. 
Use only the provided context if needed.
"""
        snippet = get_completion(
            prompt,
            max_new_tokens=max_new_tokens,
            temperature=temperature
        )
        formatted_snippet = f"### GUIDELINE #{idx}: {guideline}\n{snippet.strip()}\n"
        final_snippets.append(formatted_snippet)

    # 4) Combine all guideline-based snippets
    final_document = "\n\n".join(final_snippets)
    return final_document


In [ ]:
# ------------------------------------------------------------------------------
# EXAMPLE USAGE (Comment out if you only want library code)
# ------------------------------------------------------------------------------
if __name__ == "__main__":
    # 1) Suppose we load & chunk a sample file (PDF, TXT, or CSV)
    file_path = "sample.txt"  # adjust to your actual file
    doc_chunks = DocumentProcessor.load_and_chunk_file(file_path)

    # 2) Create a vector store and index the doc chunks
    vs = VectorStore(persist_directory="rag_index")
    vs.create_index(doc_chunks, force_recreate=True)

    # 3) Some example guidelines
    my_guidelines = [
        "Provide a step-by-step approach.",
        "Use plain language for a broad audience."
    ]

    # 4) Generate a combined doc addressing each guideline
    user_query = "How can I plan my personal finances effectively?"
    final_doc = generate_document_with_guidelines(
        user_query=user_query,
        guidelines=my_guidelines,
        vector_store=vs,
        k=3,
        max_new_tokens=512,
        temperature=0.7
    )

    print("\n===== FINAL DOCUMENT =====\n")
    print(final_doc)